# **전체 모델 구축 과정**

### **데이터 작업**
파이토치는 데이터 작업을 위해 `torch.utils.data.DataLoader` 및 `torch.utils.data.Dataset`을 제공한다.    
Dataset은 샘플과 해당 레이블을 저장하고 DataLoader는 Dataset 주위에 iterable을 래핑한다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

> 파이토치는 `TorchText`, `TorchVision`, `torchAudio`와 같은 도메일 별 라이브러리를 제공한다

**`torchvision.datasets` 모듈에는 `CIFAR`, `COCO`와 같은 많은 실제 비전 데이터에 대한 데이터 세트 개채가 포함되어있다.**

In [4]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

D:\download\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# `Dataset`을 `DataLoader`에 인수로 전달한다.
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


### **모델 제작**
파이토치에서 신경망을 정의하기 위해 nn.Module에서 상속하는 클래스를 만든다.   
`__init__` 함수에서 네트워크 계층을 정의하고 `forward` 함수에서 데이터가 네트워크를 통과하는 방법을 지정한다.

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### **모델 매개 변수 최적화**
모델을 훈련하려면 손실 함수와 최적화 프로그램이 필요하다.   
단일 학습 루프에서 모델은 학습 데이터 세트에 대한 에측을 수행하고 오류를 역전파하여 모델 매개변수를 조정한다.

In [9]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X, y
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
# 모델 성능 확인
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, corrent = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X, y
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= size
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [10]:
# 모델 학습
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------------
loss: 2.303725 [    0/60000]
loss: 2.291311 [ 6400/60000]
loss: 2.285942 [12800/60000]
loss: 2.275020 [19200/60000]
loss: 2.267132 [25600/60000]
loss: 2.269236 [32000/60000]
loss: 2.239414 [38400/60000]
loss: 2.234189 [44800/60000]
loss: 2.227993 [51200/60000]
loss: 2.219682 [57600/60000]


UnboundLocalError: local variable 'correct' referenced before assignment

In [11]:
# 모델 저장
torch.save(model.state_dict(), "data/model.pth")

# 모델 로드
model = Neuralnetwork()
model.load_state_dict(torch.load("data/model.pth"))



NameError: name 'Neuralnetwork' is not defined